# RPC

Identify appropriate UART interfaces on the server and client and wire up RX and TX. Do not forget to wire "cross", i.e. RX on one processor to TX on the other.

Optionally also wire flow control (RTS and CTS). Doing so protects from data loss (and erratic behavior) with large packages that to not fit in the UART receive buffers.

## Configure Server

In [1]:
%connect wifi-server
%rsync

Connected to wifi-server @ serial:///dev/ttyUSB0
Directories match


In [1]:
# run boot.py & main.py
%hardreset


!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   hardreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



## Configure Client

In [1]:
%connect wifi-client
%rsync
%softreset

Connected to wifi-client @ serial:///dev/ttyACM0
Directories match

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



## RPC

In [1]:
%connect wifi-client

from urpc import *
import sys
sys_ = import_('sys')

print("Platforms:")
print("   host:    {}".format(sys.platform))
print("   server:  {}".format(sys_.get_('platform')))

Connected to wifi-client @ serial:///dev/ttyACM0
Platforms:
   host:    pyboard
   server:  esp32


In [1]:
!cat code/server/rpc_test.py

class RPC_Test:

    def __init__(self, name):
        self._name = name

    def add(self, a, b):
        return "{}: {} + {} = {}".format(self._name, a, b, a+b)

    @property
    def name(self):
        return self._name

    def __str__(self):
        return "Demo {}".format(self._name)



In [1]:
rpc_test = import_('rpc_test')

t = rpc_test.RPC_Test("rpc test")
print("ADD ", t.add(5, 7))
print("NAME", t.get_('name'))
print("STR ", t)

print("\nchange name ...")
t.set_('_name', 'new name')
print("ADD ", t.add(3, -9))
print("NAME", t.get_('name'))
print("STR ", t)

ADD  rpc test: 5 + 7 = 12
NAME rpc test
STR  Demo rpc test

change name ...
ADD  new name: 3 + -9 = -6
NAME new name
STR  Demo new name


In [1]:
from urpc import *

builtins_ = import_('builtins')
builtins_.exec(
"""
help('modules')
""")

__main__          framebuf          uasyncio/lock     ure
_boot             gc                uasyncio/stream   uselect
_onewire          inisetup          ubinascii         usocket
_thread           machine           ubluetooth        ussl
_uasyncio         math              ucollections      ustruct
_webrepl          micropython       ucryptolib        usys
apa106            msgpack           uctypes           utime
btree             neopixel          uerrno            utimeq
builtins          network           uhashlib          uwebsocket
cmath             ntptime           uheapq            uzlib
dht               onewire           uio               webrepl
ds18x20           uarray            ujson             webrepl_setup
esp               uasyncio/__init__ uos               websocket_helper
esp32             uasyncio/core     upip
finaliserproxy    uasyncio/event    upip_utarfile
flashbdev         uasyncio/funcs    urandom
Plus any modules on the filesystem


In [1]:
gc_ = import_('gc')
print("1) cop mem_free: ", gc_.mem_free())
gc_.collect()
print("2) cop mem_free: ", gc_.mem_free())

import gc
print("1) host mem_free:", gc.mem_free())
gc.collect()
print("2) host mem_free:", gc.mem_free())

1) cop mem_free:  77824
2) cop mem_free:  98688
1) host mem_free: 22400
2) host mem_free: 88256
